# Mastering with ROS: Shadow Hand

## Unit 4: Smart Grasping System

<p style="background:green;color:white;">SUMMARY</p>

Estimated time of completion: <b>1h</b><br><br>
This Unit will show you how to interact and what is the purpose of the different functions integrated in the Smart Grasping System.

<p style="background:green;color:white;">END OF SUMMARY</p>

The Smart Grasping System, developed by the Shadow Robot Company, is basically a set of functions and tools which main purpose is to ease the accomplishment of grasping tasks. This functions are written in Python, and they can be found in the **smart_grasper.py** file, within the **smart_grasping_sandbox** package. During this Unit, we are going to analyze and test the functionaility of some of the main functions that this Smart Grasping System provides.

## Code explanation

Before starting to explain the different functions, let's have a quick look at the SmartGrasper class definition and initialization. Here you have the code:

In [ ]:
class SmartGrasper(object):

    __last_joint_state = None
    __current_model_name = "cricket_ball"
    __path_to_models = "/root/.gazebo/models/"
    
    def __init__(self):
        """
        This constructor initialises the different necessary connections to the topics and services
        and resets the world to start in a good position.
        """
        rospy.init_node("smart_grasper")
        
        self.__joint_state_sub = rospy.Subscriber("/joint_states", JointState, 
                                                  self.__joint_state_cb, queue_size=1)

        rospy.wait_for_service("/gazebo/get_model_state", 10.0)
        rospy.wait_for_service("/gazebo/reset_world", 10.0)
        self.__reset_world = rospy.ServiceProxy("/gazebo/reset_world", Empty)
        self.__get_pose_srv = rospy.ServiceProxy("/gazebo/get_model_state", GetModelState)

        rospy.wait_for_service("/gazebo/pause_physics")
        self.__pause_physics = rospy.ServiceProxy("/gazebo/pause_physics", Empty)
        rospy.wait_for_service("/gazebo/unpause_physics")
        self.__unpause_physics = rospy.ServiceProxy("/gazebo/unpause_physics", Empty)
        rospy.wait_for_service("/controller_manager/switch_controller")
        self.__switch_ctrl = rospy.ServiceProxy("/controller_manager/switch_controller", SwitchController)
        rospy.wait_for_service("/gazebo/set_model_configuration")
        self.__set_model = rospy.ServiceProxy("/gazebo/set_model_configuration", SetModelConfiguration)
        
        rospy.wait_for_service("/gazebo/delete_model")
        self.__delete_model = rospy.ServiceProxy("/gazebo/delete_model", DeleteModel)
        rospy.wait_for_service("/gazebo/spawn_sdf_model")
        self.__spawn_model = rospy.ServiceProxy("/gazebo/spawn_sdf_model", SpawnModel)
        
        rospy.wait_for_service('/get_planning_scene', 10.0)
        self.__get_planning_scene = rospy.ServiceProxy('/get_planning_scene', GetPlanningScene)
        self.__pub_planning_scene = rospy.Publisher('/planning_scene', PlanningScene, queue_size=10, latch=True)

        self.arm_commander = MoveGroupCommander("arm")
        self.hand_commander = MoveGroupCommander("hand")
        
        self.__hand_traj_client = SimpleActionClient("/hand_controller/follow_joint_trajectory", 
                                                     FollowJointTrajectoryAction)
        self.__arm_traj_client = SimpleActionClient("/arm_controller/follow_joint_trajectory", 
                                                    FollowJointTrajectoryAction)
                                              
        if self.__hand_traj_client.wait_for_server(timeout=rospy.Duration(4.0)) is False:
            rospy.logfatal("Failed to connect to /hand_controller/follow_joint_trajectory in 4sec.")
            raise Exception("Failed to connect to /hand_controller/follow_joint_trajectory in 4sec.")
                                              
        if self.__arm_traj_client.wait_for_server(timeout=rospy.Duration(4.0)) is False:
            rospy.logfatal("Failed to connect to /arm_controller/follow_joint_trajectory in 4sec.")
            raise Exception("Failed to connect to /arm_controller/follow_joint_trajectory in 4sec.")
            
        self.reset_world()

Within this code, several things are being done:

1. First of all, we are initializing some variables, which will be used by the different functions. This variable are **__last_joint_state**, **__current_model_name** and **__path_to_models**.
<br><br>
2. Second, inside the constructor of the class, we are creating all the necessary connections to the different topics and services of the simulation.
<br><br>
3. Finally, at the end of the constructor, we call the **reset_world()** function, which will reset the world in order to start in the proper position.

Great! So with the proper introductions made, let's start to check the main functions!

### reset_world

The first function we find in the code, is the **reset_world()** function. It is used to reset the object poses in the world and the robot joint angles. So, to set the world to an initial state. Here's the code.

In [ ]:
def reset_world(self):

    self.__switch_ctrl.call(start_controllers=[], 
                            stop_controllers=["hand_controller", "arm_controller", "joint_state_controller"], 
                            strictness=SwitchControllerRequest.BEST_EFFORT)
    self.__pause_physics.call()

    joint_names = ['shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint', 
                   'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']
    joint_positions = [1.2, 0.3, -1.5, -0.5, -1.5, 0.0]

    self.__set_model.call(model_name="smart_grasping_sandbox", 
                          urdf_param_name="robot_description",
                          joint_names=joint_names, 
                          joint_positions=joint_positions)

    timer = Timer(0.0, self.__start_ctrl)
    timer.start()

    time.sleep(0.1)
    self.__unpause_physics.call()

    self.__reset_world.call()

In order to use it, you will have to use the following structure:

In [ ]:
from smart_grasping_sandbox.smart_grasper import SmartGrasper
import time

sgs = SmartGrasper()
sgs.reset_world()

So... what are you doing here? Very easy! 
<br><br>
1. First of all, you are importing the **SmartGrasper** class from the **smart_grasper.py** file. 
<br>
2. Then, you import the **time** module, because it will be used inside the **reset_world()** function. 
<br>
3. Next, you create an object of the SmartGrasper class.
<br>
4. And finally, you call the reset_world() function.

This is the structure you will have to use to call any of the functions of the SmartGrasper class.

### get_object_pose

The next function is the **get_object_pose()**. The purpose of this function is very clear. Return the pose of the object to grasp.

In [ ]:
def get_object_pose(self):
        
    return self.__get_pose_srv.call(self.__current_model_name, "world").pose

As you can see in the code, it is getting the pose of the object stated in the **__current_model_name** variable, which as you can see in the initialization of the class, it is set to **cricket_ball**, which is the name in the simulation of the red ball.

So, in order to use this function, you will have to use:

In [ ]:
from smart_grasping_sandbox.smart_grasper import SmartGrasper

sgs = SmartGrasper()
sgs.get_object_pose()

### get_tip_pose

The **get_tip_pose()** function is also very simple and clear. It will return the pose of the end-effector, which in this case is the Shadow Hand itself.

In [ ]:
def get_tip_pose(self):

    return self.arm_commander.get_current_pose(self.arm_commander.get_end_effector_link()).pose

<p style="background:#EE9023;color:white;">Exercise 4.1</p>

a) Inside your workspace, create a new folder (**NOT A ROS PACKAGE**) called **my_grasping_scripts**.

b) Inside this folder, write a simple Python script that will call the **get_object_pose** and **get_tip_pose** functions, and print the result in the screen.

c) If, for instance, you name your file **exercise1.py**, in order to execute it, you can type the following command:

In [ ]:
python my_script.py

You should get something similar to this:

<img src="img/ex_4_1.png" width="800" />

<p style="color:red;">**NOTE**: Don't worry about the error that appears at the end of the output. This is a known issue. It is caused by certain launched processes that were not closed correctly, but it doesn't has any effect on the execution of the code itself. This error should be fixed for the newest ROS distributions, like Kinetic.</p>

<p style="background:#EE9023;color:white;">End of Exercise 4.1</p>

### move_tip_absolute

The next function we find is the **move_tip_absolute()**. Here is the code:

In [ ]:
def move_tip_absolute(self, target):

    self.arm_commander.set_start_state_to_current_state()
    self.arm_commander.set_pose_targets([target])
    plan = self.arm_commander.plan()
    if not self.arm_commander.execute(plan):
        return False
    return True

As you can see, this function gets as input a **target** variable. This variable is a **geometry_msgs/Pose** message. The function, then, moves the Shadow Hand to that poisition in the world frame.

### move_tip

We also have the **move_tip()** function. The difference between this function and the previous one, is that in this one, you pass directly to the function the **(x, y, z, yaw, pitch, roll)** values that you want to move your Shadow Hand. Here you have the code.

In [ ]:
def move_tip(self, x=0., y=0., z=0., roll=0., pitch=0., yaw=0.):

    transform = PyKDL.Frame(PyKDL.Rotation.RPY(pitch, roll, yaw),
                            PyKDL.Vector(-x, -y, -z))

    tip_pose = self.get_tip_pose()
    tip_pose_kdl = posemath.fromMsg(tip_pose)
    final_pose = toMsg(tip_pose_kdl * transform)

    self.arm_commander.set_start_state_to_current_state()
    self.arm_commander.set_pose_targets([final_pose])
    plan = self.arm_commander.plan()
    if not  self.arm_commander.execute(plan):
        return False
    return True

<p style="background:#EE9023;color:white;">Exercise 4.2</p>

a) Using the **move_tip** function, create a Python script that moves the Shadow Hand 0.15 meters down in the y axis.

You should get something like this:

<img src="img/move_tip.gif" width="600" />

<p style="background:#EE9023;color:white;">End of Exercise 4.2</p>

### send_command

The next function we are going to check is the **send_command()** function. Here is the code:

In [ ]:
def send_command(self, command, duration=0.2):

    hand_goal = None
    arm_goal = None

    for joint, target in command.items():
        if "H1" in joint:
            if not hand_goal:
                hand_goal = FollowJointTrajectoryGoal()

                point = JointTrajectoryPoint()
                point.time_from_start = rospy.Duration.from_sec(duration)

                hand_goal.trajectory.points.append(point)

            hand_goal.trajectory.joint_names.append(joint)
            hand_goal.trajectory.points[0].positions.append(target)
        else:
            if not arm_goal:
                arm_goal = FollowJointTrajectoryGoal()

                point = JointTrajectoryPoint()
                point.time_from_start = rospy.Duration.from_sec(duration)

                arm_goal.trajectory.points.append(point)

            arm_goal.trajectory.joint_names.append(joint)
            arm_goal.trajectory.points[0].positions.append(target)

    if arm_goal:
        self.__arm_traj_client.send_goal_and_wait(arm_goal)
    if hand_goal:
        self.__hand_traj_client.send_goal_and_wait(hand_goal)

As you can see, this function gets as input a **command** variable. This variable is a dictionary that contains **joint_names** associated with a **target** value. Something like this:

In [ ]:
{"H1_F1J1": -1.0, "shoulder_pan_joint": 1.0}

The values for each joint_name will define a trajectory, which will then be executed, if correct, by the robot.

<p style="background:#EE9023;color:white;">Exercise 4.3</p>

a) Using the **send_command** function, create a Python script that moves the robot to a position like to this one:

<img src="img/cmd_position.png" width="600" />

The values of the joints in this position are the following:

<img src="img/cmd_joint_values.png" width="800" />

<p style="background:#EE9023;color:white;">End of Exercise 4.3</p>

### get_current_joint_state

This function is also very easy and clear. It simply returns the current values of the joints of the robot.

In [ ]:
def get_current_joint_state(self):

    joints_position = {n: p for n, p in
                       zip(self.__last_joint_state.name,
                           self.__last_joint_state.position)}
    joints_velocity = {n: v for n, v in
                       zip(self.__last_joint_state.name,
                       self.__last_joint_state.velocity)}
    joints_effort = {n: v for n, v in
                     zip(self.__last_joint_state.name, 
                     self.__last_joint_state.effort)}
    return joints_position, joints_velocity, joints_effort

### open_hand

The **open_hand()**, as the names itself states, it just opens the Shadow Hand, in order to prepare it for a Grasping action. Here you can check the code:

In [ ]:
def open_hand(self):

        self.hand_commander.set_named_target("open")
        plan = self.hand_commander.plan()
        if not self.hand_commander.execute(plan, wait=True):
            return False

        return True

### close_hand

Just as the previous function, the **close_hand()** function just closes the Shadow Hand.

In [ ]:
def close_hand(self):

    self.hand_commander.set_named_target("close")
    plan = self.hand_commander.plan()
    if not self.hand_commander.execute(plan, wait=True):
        return False

    return True

<p style="background:#EE9023;color:white;">Exercise 4.4</p>

a) Using the **open_hand()** and **close_hand()** functions, create a simple Python script that opens and closes the Shadow Hand two times each.

You should get something like this:

<img src="img/open_close_hand.gif" width="600" />

<p style="background:#EE9023;color:white;">End of Exercise 4.4</p>

### check_fingers_collisions

The next function is the **check_fingers_collision()**. Here you can see the code:

In [ ]:
def check_fingers_collisions(self, enable=True):

    objects = ["cricket_ball__link", "drill__link", "cafe_table__link"]

    while self.__pub_planning_scene.get_num_connections() < 1:
        rospy.loginfo("waiting for someone to subscribe to the /planning_scene")
        rospy.sleep(0.1)

    request = PlanningSceneComponents(components=PlanningSceneComponents.ALLOWED_COLLISION_MATRIX)
    response = self.__get_planning_scene(request)
    acm = response.scene.allowed_collision_matrix

    for object_name in objects:
        if object_name not in acm.entry_names:
            # add object to allowed collision matrix
            acm.entry_names += [object_name]
            for row in range(len(acm.entry_values)):
                acm.entry_values[row].enabled += [False]

            new_row = deepcopy(acm.entry_values[0])
            acm.entry_values += {new_row}

    for index_entry_values, entry_values in enumerate(acm.entry_values):
        if "H1_F" in acm.entry_names[index_entry_values]:
            for index_value, _ in enumerate(entry_values.enabled):
                if acm.entry_names[index_value] in objects:
                    if enable:
                        acm.entry_values[index_entry_values].enabled[index_value] = False
                    else:
                        acm.entry_values[index_entry_values].enabled[index_value] = True
        elif acm.entry_names[index_entry_values] in objects:
            for index_value, _ in enumerate(entry_values.enabled):
                if "H1_F" in acm.entry_names[index_value]:
                    if enable:
                        acm.entry_values[index_entry_values].enabled[index_value] = False
                    else:
                        acm.entry_values[index_entry_values].enabled[index_value] = True

    planning_scene_diff = PlanningScene(is_diff=True, allowed_collision_matrix=acm)
    self.__pub_planning_scene.publish(planning_scene_diff)
    rospy.sleep(1.0)

If you call this function with the enable parameter to True, it will check for possible collisions between the fingers of the SHadow Hand and the table or the objects.

In [ ]:
sgs.check_fingers_collisions(True)

If you call this function with the enable argument to False, it will disable this collision checking.

In [ ]:
sgs.check_fingers_collisions(False)

### pick

The **pick()** function is one of the most important ones, because it contains many of the functions we've seen inside it. By calling to this function, you will start a complete pick action. This is the function you called in the first Demo you did in the Introdution the Course! Here you have the code:

In [ ]:
def pick(self):

    rospy.loginfo("Moving to Pregrasp")
    self.open_hand()
    time.sleep(0.1)

    ball_pose = self.get_object_pose()
    ball_pose.position.z += 0.5

    #setting an absolute orientation (from the top)
    quaternion = quaternion_from_euler(-pi/2., 0.0, 0.0)
    ball_pose.orientation.x = quaternion[0]
    ball_pose.orientation.y = quaternion[1]
    ball_pose.orientation.z = quaternion[2]
    ball_pose.orientation.w = quaternion[3]

    self.move_tip_absolute(ball_pose)
    time.sleep(0.1)

    rospy.loginfo("Grasping")
    self.move_tip(y=-0.164)
    time.sleep(0.1)
    self.check_fingers_collisions(False)
    time.sleep(0.1)
    self.close_hand()
    time.sleep(0.1)

    rospy.loginfo("Lifting")
    for _ in range(5):
        self.move_tip(y=0.01)
        time.sleep(0.1)

    self.check_fingers_collisions(True)

Let's analyze part by part what this functions does:

1- First of all, it opens the Shadow Hand.

In [ ]:
rospy.loginfo("Moving to Pregrasp")
self.open_hand()
time.sleep(0.1)

2- Next, it gets the position of the object to grasp, and adds to the z axis 0.5 meters, so that the Shadow Hand will go right above the object.

In [ ]:
ball_pose = self.get_object_pose()
ball_pose.position.z += 0.5

3- Next, it makes sure that the Shadow Hand will be orientated facing the table. It does this by setting the corresponding orientation value, and the calling the **move_tip_abosulte()** function to apply this orientation to the Shadow Hand.

In [ ]:
quaternion = quaternion_from_euler(-pi/2., 0.0, 0.0)
ball_pose.orientation.x = quaternion[0]
ball_pose.orientation.y = quaternion[1]
ball_pose.orientation.z = quaternion[2]
ball_pose.orientation.w = quaternion[3]

self.move_tip_absolute(ball_pose)

4- Next, it moves the Shadow Hand a little bit down using the **move_tip** function.

In [ ]:
self.move_tip(y=-0.164)

4- It disable the collision checking, because the fingers are going to be in collision with the object to grasp when it closes the hand.

In [ ]:
self.check_fingers_collisions(False)

5- And it closes the hand in order to catch the ball.

In [ ]:
self.close_hand()

6- Finally, it moves the Shadow Hand up again, in order to lift a little bit the ball.

In [ ]:
for _ in range(5):
    self.move_tip(y=0.01)
    time.sleep(0.1)

So... what do you say? Now that you know how this function works, and what are the purposes of each functions, it doesn't look that complicated, right?

And that's it! In the Next Unit, you will learn how to use Object Recognition in order to get the position of the object you want to grasp, without getting from the simulation data (like it is being donde now).